## mistral-unslothify

Use 🤗huggingface and Unsloth to finetune a Mistral pre-trained model on domain data.

The aim here is to fine-tune the Mistral pre-trained model based on a set of text items that will attune it to perform more effectively on a specific domain. This means that we will be adapting the general capabilities of the model to better suit the needs and nuances of a particular field or area of expertise.

Maybe surprisingly, in spite of a model such as for example Mistral 7B having seven billion parameters, the number of domain examples needed to fine-tune it can often be relatively small (see, for example, the widely cited paper on LLMs being ['few-shot learners'](https://arxiv.org/abs/2005.14165)). This is because large models like Mistral have a robust foundation of general language understanding, and the fine-tuning acts more like a focused nudge refining what the model already knows, rather than starting from scratch.

As an example here, we have the file `domain.jsonl` with 500 text items to fine-tune the model for. All examples are about burgers 🍔🍔🍔, so the model will become specifically tailored to text-based tasks around burgers.

<img align="right" src="icons/mistral.png" width="60" hspace="10"> 
<img align="right" src="icons/hf.png" width="60" hspace="10"> 
<img align="right" src="icons/unsloth.png" width="150" hspace="10">

<hr>
**NOTE**<br>
This whole thing must be run on GPU. Either on a local machine with Nvidia/Cuda properly installed, on Google Colab with a free GPU runtime (even though they quickly run out), or any other cloud machine where the `!nvcc --version` cell below checks out ✅.

## 1. Setup and installations

Check to see that we have a GPU and Cuda driver.

In [1]:
# check cuda version
!nvcc --version

zsh:1: command not found: nvcc


In [5]:
!pip install --upgrade pip -q

# install the latest closest available cuda build
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.9 MB/s eta 0:00:00


Download `unsloth` from Github and install it.

In [4]:
!git clone https://github.com/unslothai/unsloth.git
!cd unsloth && pip install . -q
#!pip show unsloth

fatal: destination path 'unsloth' already exists and is not an empty directory.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Install remaining needed libraries.

In [15]:
!pip install numpy -q
!pip install bitsandbytes -q
!pip install unsloth-zoo -q
!pip install xformers -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Imports

In [23]:
import torch
import json
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TrainingArguments
from trl import SFTTrainer

## 2. Download the pre-trained model and prepare for fine-tuning

Download pre-trained model

In [55]:
model_name = "unsloth/mistral-7b-instruct-v0.3"

In [56]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth 2024.11.7: Fast Mistral patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


#### Add LoRA adapters

In [57]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    #use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

## 3. Prepare fine-tuning data



In [58]:
jsonl_file = "domain.jsonl"

Load and reformat the domain data.

In [59]:
# use the dataset loader by Huggingface and some formatting functions
dataset = load_dataset("json", data_files=jsonl_file, split="train")

tokenizer.pad_token = tokenizer.eos_token

def format_text(examples):
    texts = [note + tokenizer.pad_token for note in examples["text"]]
    return {"text": texts}

dataset = dataset.map(format_text, batched=True)

Tokenize the domain data with the pre-trained model's tokenizer.

In [60]:
# Initialize the tokenizer
#tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the text
def tokenize_texts(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_texts, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [61]:
# Remove unneeded columns and set format for PyTorch
tokenized_dataset = tokenized_dataset.remove_columns(["text"])  # Keep only tokenized columns
tokenized_dataset.set_format(type="torch")

## 4. Fine-tune the model

We want the *training loss* to decrease. A loss value around 2-3 is reasonable, if it gets close to 1.0 or drop below the predictions will be highly confident, but also with some risk of overfitting, meaning that the model has learned the training data too well and may not perform as effectively on unseen data.

In [1]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    max_seq_length=1024,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2, # ADJUST BASED ON AVAILABLE MEM
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=1, # NUMBER OF FULL PASSES THROUGH THE TRAINING DATASET
        learning_rate=2e-4, # LOWER VALUES = SLOW BUT MORE STABLE LEARNING
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01, # APPLIES L2 REGULARIZATION TO AVOID OVERFITTING
        lr_scheduler_type="linear",
        seed=1337,
        output_dir="outputs",
        report_to="none",
    ),
)

trainer_stats = trainer.train()

ModuleNotFoundError: No module named 'trl'